In [4]:
import numpy as np

In [72]:
graph = np.loadtxt('./input/complicatedGraph.csv',delimiter=',')

In [73]:
graph

array([[0.  , 0.35, 0.  , 0.  , 0.  , 1.  ],
       [0.45, 0.3 , 0.  , 0.  , 0.  , 0.  ],
       [0.3 , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 1.  , 0.  , 0.  , 0.  ],
       [0.25, 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.35, 0.  , 0.  , 0.  , 0.  ]])

To be an ergodic graph, consequently to be able to perform the pagerank algorithm, the graph we are evaluating must be: 

* Stochastic (every column's outgoing probability sums to 1)
* Aperiodic: interval between two visits to the same state shouldn't be fixed
* Irreducible: from any state there is non-zero probability of going from any one state to any other

In [70]:
def pagerank(matrix, initial=None): 
    initial = np.ones(len(graph[0]))/len(graph[0]) if initial is None else initial
    qi = initial
    for _ in range(100):
        qi = np.dot(matrix, qi);
    return qi

In [71]:
pagerank(graph)

array([9.41732590e-13, 9.01473577e-13, 3.03030303e-01, 3.03030303e-01,
       3.05718980e-13, 4.09709316e-13])

The reason pagerank chokes on this graph is that edge e is a dead-end, so unless the algorithm got stuck alternating between c and d (spider-trap), it's importance gets sucked up by the dead end.

In [74]:
def pagerank_teleportation(matrix, initial=None, beta=0.85):
    # This will set up initial probabilities which will "kick-start" page-rank    
    initial = np.ones(len(graph[0]))/len(graph[0]) if initial is None else initial
    qi = initial
    
    for _ in range(100):
        # The equation here uses random teleportation to make the 
        qi = np.dot((beta)*matrix + (1-beta)*(np.ones_like(matrix)/len(graph[0])), qi);
    return qi

In [75]:
pagerank_teleportation(graph)

array([0.00119747, 0.00106529, 0.00417822, 0.00400938, 0.00056948,
       0.00063464])

We are making progress here, we added a Irreducible factor into the graph. However, the dead end is still disruptive to analysis becasue it is sucking the importance out of the graph

In [76]:
def pagerank_teleportation_stochastic(matrix, initial=None, beta=0.8):
    matrix = matrix.copy()
    # This will set up initial probabilities which will "kick-start" page-rank    
    initial = np.ones(len(graph[0]))/len(graph[0]) if initial is None else initial
    qi = initial
    
    # We will take care of dead-ends here
    sum_probs = np.sum(graph, axis=0);
    for column_index in range(len(graph[0])):
        # Complete dead end
        if (sum_probs[column_index] == 0):
            graph[:,column_index] = initial;
            print("changed column {} as following: ".format(column_index));
            print(graph);
        # Leaky dead end
        elif (sum_probs[column_index] != 1):
            graph[:,column_index] = graph[:,column_index] / sum_probs[column_index];
            print("changed column {} as following: ".format(column_index));
            print(graph);
            
    for _ in range(100):
        # The equation here uses random teleportation to address spider traps
        qi = np.dot((beta)*matrix + (1-beta)*(np.ones_like(matrix)/len(graph[0])), qi);
    return qi

In [77]:
pagerank_teleportation_stochastic(graph)

changed column 4 as following: 
[[0.         0.35       0.         0.         0.16666667 1.        ]
 [0.45       0.3        0.         0.         0.16666667 0.        ]
 [0.3        0.         0.         1.         0.16666667 0.        ]
 [0.         0.         1.         0.         0.16666667 0.        ]
 [0.25       0.         0.         0.         0.16666667 0.        ]
 [0.         0.35       0.         0.         0.16666667 0.        ]]


array([0.00070412, 0.00062276, 0.00186057, 0.00176407, 0.00034717,
       0.00038244])

We have not entirely alliviated the spider trap problem, but that can easily adjusted by changing our beta parameter:

In [78]:
pagerank_teleportation_stochastic(graph, beta=0.6)

array([0.17394094, 0.15114286, 0.24135362, 0.22178526, 0.10306423,
       0.10871309])

Notice that if we disabled teleportation, we would get stuck in a spider trap, that is a set of nodes which alternate states but there is no way to get out of the circuit

In [79]:
pagerank_teleportation_stochastic(graph, beta=1)

array([1.43277160e-08, 1.33921293e-08, 4.99999981e-01, 4.99999979e-01,
       5.26524059e-09, 6.57027397e-09])

As stated above, essentially all of the matrix rank resides with nodes c and d